# Can Language Models produce correct structured output?
The goaal of this notebook is to explorer the nature of the mistakes a Language Modell can make when producing structured output like code. The idea is that it is prone to making syntactic errors that could be avoided when scaling up the dataset.

In [2]:
from src.RawDataLoaders import CodeSearchNet_RawDataLoader
from src.models_and_transforms.text_transforms import *
from src.models_and_transforms.BART_models import *
from src.pipe_datasets import *

from pytorch_lightning import Trainer, Callback, seed_everything
import torch
import ray
ray.init(ignore_reinit_error=True)
import random
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pickle

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

2020-09-03 08:03:07,293	ERROR worker.py:666 -- Calling ray.init() again after it has already been called.


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [4]:
code_search_net_RawDataLoader = CodeSearchNet_RawDataLoader("./datasets/code_search_net", language="python", max_chars=1000)

In [5]:
samples = code_search_net_RawDataLoader.get_samples("train", fields=['code'])

In [6]:
train_dataset = BART_Span_Prediction_dataset(samples)

In [13]:
my_train_dataloader = train_dataset.to_dataloader(2)

In [11]:
model = BART_Simple()

In [19]:
lengths = [len(train_dataset[i]['input_ids']) for i in tqdm(range(10000))]

In [26]:
trainer = Trainer(gpus=1, gradient_clip_val=0.5, amp_level='O1')
trainer.fit(model, train_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name | Type                         | Params
------------------------------------------------------
0 | BART | BartForConditionalGeneration | 406 M 
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 32 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [14]:
torch.save({"model":model, "train_dataset":train_dataset}, "./BART_codesearchnet_untrained.torch")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BART_Simple. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BartForConditionalGeneration. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BartModel. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Embedding. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "

In [3]:
load_obj = torch.load("./BART_codesearchnet_untrained.torch")
model = load_obj['model']
train_dataset = load_obj['train_dataset']
train_dataloader = train_dataset.to_dataloader(2, num_workers=8)

trainer = Trainer(gpus=1, gradient_clip_val=0.5, amp_level='O1')
trainer.fit(model, train_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]

  | Name | Type                         | Params
------------------------------------------------------
0 | BART | BartForConditionalGeneration | 406 M 


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:25: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1

In [8]:
torch.save(my_train_dataloader.dataset.real_time_pipe[1], "./numericailzer.torch")

In [9]:
numericalilzer = torch.load("./numericailzer.torch")

In [10]:
numericalilzer.numericalizer.encode("this is a test")

[0, 9226, 16, 10, 1296, 2]